<a href="https://colab.research.google.com/github/LucasVD23/T1_AM1/blob/main/Testes_do_T1_de__AM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Trabalho 1 de AM1**

**Integrantes**

Lucas Vinícius Domingues 769699

Rafael Yoshio Yamawaki Murata 769681

Victor Luís Aguilar Antunes 769734

**Link para o Dataset utilizado: https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Coimbra**

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn import metrics

***Carregar o dataset no colab!***

**Atenção!**

*   Cada teste leva aproximadamente 15 minutos para ser terminado
*   O teste com SelectKBest está em um arquivo avulso no repositório Git devido ao seu tamanho e tempo de execuções elevados (aproximadamente 1 hora para executar)



In [24]:
data_cancer = shuffle(pd.read_csv("/content/dataR2.csv"),random_state = 15)
corrs = data_cancer.corr()['Classification']

In [25]:
data_cancer.head()

,Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1,Classification
11,29,23.010000,82,5.663,1.145436,35.5900,26.720000,4.58000,174.800,1
107,46,33.180000,92,5.750,1.304867,18.6900,9.160000,8.89000,209.190,2
106,45,29.384757,90,4.713,1.046286,23.8479,6.644245,15.55625,621.273,2
100,74,28.650138,88,3.012,0.653805,31.1233,7.652220,18.35574,572.401,2
25,45,37.035608,83,6.760,1.383997,39.9802,4.617125,8.70448,586.173,1


In [26]:
dtc_params = {'criterion': ['entropy','gini'],
              'max_depth': range(2,30,2),
              'min_samples_leaf':range(2,10,2)}

rfc_params = {'criterion': ['entropy','gini'],
              'max_depth': range(2,30,2),
              'min_samples_leaf':range(2,10,2),
              'bootstrap':[False,True],
              }
knnc_params = {'n_neighbors': range(2,15,1),
               'weights' : ['uniform', 'distance'],
               'metric':['minkowski','euclidean','manhattan']}
svmc_params = {'kernel' : ['linear','rbf','poly'],
               'C': np.arange(0.25,2,0.01)}
adac_params = {'n_estimators' : range(1,200,5),
               'learning_rate': np.arange(0.1,2,0.1)}

In [27]:
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
knnc = KNeighborsClassifier()
svmc = svm.SVC()
adac = AdaBoostClassifier()

In [28]:
classifiers = {'Decision Tree': dtc, 'RandomForest':rfc,'KNN':knnc,'SVM':svmc, 'AdaBoost':adac}
parametros = {'Decision Tree': dtc_params, 'RandomForest': rfc_params, 'KNN':knnc_params, 'SVM':svmc_params, 'AdaBoost': adac_params}

In [29]:
fb_scorer = metrics.make_scorer(metrics.fbeta_score,beta=1.5)

In [30]:
kf = KFold(n_splits=5,shuffle = False)

# **Pra utilizar todos os atributos, rode este bloco e depois rode "Testar Modelos"**

In [31]:
previsores = np.array(data_cancer.iloc[:,0:9].values)
classes = np.array(data_cancer.iloc[:,9:10].values)

# **Para utilizar os atributos descritos no artigo, execute este bloco e depois rode "Testar Modelos"**

In [ ]:
data_cancer = data_cancer.drop(columns=['Insulin','HOMA','Leptin','Adiponectin', 'MCP.1'])

In [ ]:
previsores = np.array(data_cancer.iloc[:,0:4].values)
classes = np.array(data_cancer.iloc[:,4:5].values)

# **Para testar os 4 atributos com maior correlação de Pearson com o diagnóstico,rode este bloco e depois rode "Testar Modelos"**

In [ ]:
data_cancer = data_cancer.drop(columns=['Age','BMI','Leptin','Adiponectin', 'MCP.1'])

In [ ]:
previsores = np.array(data_cancer.iloc[:,0:4].values)
classes = np.array(data_cancer.iloc[:,4:5].values)

# **Testar Modelos**

In [32]:
scaler = MinMaxScaler(feature_range=(0,1))
previsores = scaler.fit_transform(previsores)
classes = np.reshape(classes, (classes.shape[0],))

In [33]:
for i in range(0,len(classes)):
    if(classes[i] == 1 ):
        classes[i] = 0
    else:
        classes[i] = 1
    

In [34]:
best = {}
for i in classifiers:
  gs = GridSearchCV(classifiers[i],param_grid=parametros[i],scoring = fb_scorer,cv=kf.get_n_splits(previsores))
  gs.fit(previsores,classes)
  best[i] = [gs.best_estimator_,gs.best_score_]


KeyboardInterrupt: ignored

In [ ]:
#RESULTADOS DO TESTE
for i in best:
  print(best[i])

In [ ]:
cfs_fbetas = {}
for i in classifiers:
  print("-----------------------" + i + "---------------------")
  cf = best[i][0]
  acuracias = []
  revocs = []
  fb_scores =[]
  precisao=[]
  especificidade=[]
  auc=[]

  for train_index, test_index in kf.split(previsores):
      X_train, X_test = previsores[train_index], previsores[test_index]
      y_train, y_test = classes[train_index], classes[test_index]
      
      y_train = np.reshape(y_train, (y_train.shape[0],))
      cf.fit(X_train, y_train)
      y_pred = cf.predict(X_test)

      TN, FP, FN, TP = metrics.confusion_matrix(y_test, y_pred).ravel()

      #Acuracia — (TP + TN) / total
      acuracias.append(metrics.accuracy_score(y_test,y_pred))
      #Revocaçao — TP / (TP + FN)
      revocs.append(metrics.recall_score(y_test,y_pred))
      #Precissao — TP / (TP + FP)
      precisao.append(metrics.precision_score(y_test,y_pred))
      #Especificidade — TN / (TN + FP)
      especificidade.append(TN / (TN + FP))
      
      fb_scores.append(metrics.fbeta_score(y_test,y_pred,1.5))
      auc.append(metrics.roc_auc_score(y_test, y_pred))

  print("média das acuracias: " + str(np.mean(acuracias)))
  print("média das revocações: " + str(np.mean(revocs)))
  print("média das precisões: " + str(np.mean(precisao)))
  print("média das especificidades: " + str(np.mean(especificidade)))
  print("média das f_betas: " + str(np.mean(fb_scores)))
  print("média das AUCs: " + str(np.mean(auc)))

  cfs_fbetas[i] = np.mean(fb_scores)


#EDA 

Idade: atributo numérico discreto, o resto (tirando a classe) é contínuo o resto é ratio

Atributo Alvo = 1 para paciente saudável, 2 para paciente com câncer


**Sumarização**


In [ ]:
for coluna in data_cancer.columns[0:9]:
  print("---------------- " + coluna + " ----------------")
  print("MAX: " + str(data_cancer[coluna].max()))
  print("MIN: " + str(data_cancer[coluna].min()))
  print("MÉDIA: " + str(data_cancer[coluna].mean()))
  print("DESV PADRÃO: "+ str(data_cancer[coluna].std()))
  print("Correlação: "  + str(corrs[coluna]))

---------------- Age ----------------
MAX: 89
MIN: 24
MÉDIA: 57.30172413793103
DESV PADRÃO: 16.112765572452286
Correlação: -0.04355468131279905
---------------- BMI ----------------
MAX: 38.57875854
MIN: 18.37
MÉDIA: 27.582110827413793
DESV PADRÃO: 5.020135768568436
Correlação: -0.13258574047668428
---------------- Glucose ----------------
MAX: 201
MIN: 60
MÉDIA: 97.79310344827586
DESV PADRÃO: 22.525161676350454
Correlação: 0.3843153767484967
---------------- Insulin ----------------
MAX: 58.46
MIN: 2.432
MÉDIA: 10.012086206896551
DESV PADRÃO: 10.067767636993999
Correlação: 0.2768037140962369
---------------- HOMA ----------------
MAX: 25.05034187
MIN: 0.46740866700000006
MÉDIA: 2.694987564396552
DESV PADRÃO: 3.6420431918896896
Correlação: 0.2840122224119574
---------------- Leptin ----------------
MAX: 90.28
MIN: 4.311
MÉDIA: 26.615080172413794
DESV PADRÃO: 19.18329407226946
Correlação: -0.001078456594852485
---------------- Adiponectin ----------------
MAX: 38.04
MIN: 1.65602
MÉDIA: 

**Testando Histogramas**

In [ ]:
for coluna in data_cancer.columns[0:9]:
    plt.hist(data_cancer[coluna],density = False)
    plt.title(coluna)
    plt.show()

In [ ]:

plt.hist(data_cancer['Age'],density = False,bins = range(20,90,10),histtype='bar')
plt.title('Age')
plt.savefig('HistAge.png', dpi = 500)
plt.show()

**Testando Boxplots**

In [ ]:
for coluna in data_cancer.columns[0:9]:
    plt.boxplot(data_cancer[coluna])
    plt.title(coluna)
    plt.savefig((coluna+"boxplot.png"),dpi = 500)
    plt.show()

**Testando Relações entre diagnósticos e Parâmetros**

In [ ]:
for coluna in data_cancer.columns[0:9]:
    my_dict = {'Negativo': data_cancer[data_cancer['Classification'] == 1][coluna], 
               'Positivo': data_cancer[data_cancer['Classification'] == 2][coluna]}
    
    fig, ax = plt.subplots()
    ax.boxplot(my_dict.values())
    ax.set_xticklabels(my_dict.keys())
    plt.title('Relação ' + coluna + ' X Diagnóstico')
    plt.savefig(coluna+'relDiagnostico.png',dpi=500)
   
